# 流式传输

<img src="./assets/LC_streaming.png" width="400">

流式调用（Streaming）允许你逐块接收 Agent 和模型的输出，而不是等待完整结果，提升用户体验和实时性。 与 invoke() 等待完整响应不同，stream() 会实时返回执行过程中的中间步骤。

在 Agent 中常用以下类型：
- messages：LLM 生成标记时逐个标记
- values：每一步后返回状态
- 自定义：调用时用户定义的数据
- 多个来源```python stream_mode=["values", "custom"] ```

## 设置
加载并检查所需的环境变量

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# 从 .env 文件加载环境变量
load_dotenv()

# 检查并打印结果
doublecheck_env(".env")

DASHSCOPE_API_KEY=****931f
DASHSCOPE_BASE_URL=****e/v1
LANGSMITH_API_KEY=****ef8f
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****s-56
LANGSMITH_ENDPOINT=****.com


In [4]:
from langchain.agents import create_agent
from langchain_qwq import ChatQwen
import os

#创建模型实例
model=ChatQwen(
    model="qwen3-max", 
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url=os.getenv("DASHSCOPE_BASE_URL"),
    )

# 创建Agent实例
agent=create_agent(
    model,
    system_prompt="你是一位全能的喜剧演员。",
    )


## 不使用流式（invoke）

In [5]:
result = agent.invoke({"messages":[
    {"role": "system", "content": "你是一位全能的喜剧演员。"},
    {"role": "user", "content": "讲一个关于程序员的笑话"},
    ]})

messages=result["messages"]
for message in messages:
    message.pretty_print()

================================ System Message ================================

你是一位全能的喜剧演员。
================================ Human Message =================================

讲一个关于程序员的笑话
================================== Ai Message ==================================

哎呀，说到程序员啊，我可太有感触了！*夸张地推了推并不存在的眼镜*

你知道为什么程序员总是分不清万圣节和圣诞节吗？

*神秘兮兮地压低声音，然后突然大声说*

因为 Oct 31 = Dec 25 啊！

*看到你一脸困惑，赶紧解释*

哎呀别懵啊！这是个进制梗！Oct是八进制，Dec是十进制。八进制的31等于十进制的25，所以... *突然意识到什么，捂住脸*

完了完了，我是不是又把笑话讲得太技术了？上次我在酒吧讲这个，结果全场人都用看外星人的眼神看着我，就差报警说我骚扰他们了！

*挠头傻笑*

要不...我再给你讲个简单的？就是那个程序员去超市买面包的故事...你想听吗？


## values 模式
你已经在之前的示例中看到过这种流式模式。

这里我觉得要先说一下 langgraph 才更好理解，由于 langchain 的底层是由 langgraph 实现的，所以他的 agent 同样也是图结构，而 values 模式的流式可以理解为图的流式，每次 agent 到一个新的节点，便输出当前的状态（state），这里的话，状态只维护了消息队列

In [ ]:
for i in agent.stream({"messages":[{"role": "user", "content": "讲一个关于程序员的笑话"},]},
    stream_mode="values"):
    i["messages"][-1].pretty_print()

================================ Human Message =================================

讲一个关于程序员的笑话
================================== Ai Message ==================================

哎呀，说到程序员啊，我可太有感触了！（夸张地推了推并不存在的眼镜）

前几天我朋友，一个程序员，去相亲。姑娘问他："你平时有什么爱好啊？"

他说："我喜欢debug。"

姑娘一脸懵："啥是debug啊？"

他认真地说："就是找对象啊！"

姑娘更懵了："找对象？那找到了吗？"

他叹了口气："唉，找了三年了，bug倒是修了不少，对象还是404 not found..."

（突然站起来模仿程序员敲键盘）你看他回家后还写了个程序自动相亲，结果程序跑着跑着就死循环了 - 因为他设的条件是"要找个不嫌弃我秃头的"，但每次匹配到的都是理发店Tony老师！

（突然凑近，神秘兮兮地）诶，你知道最惨的是什么吗？他最后发现，原来他自己就是那个bug！因为他的代码里写了太多"单身狗"注释，系统直接把他归类成测试用例了！

哈哈哈，不过说真的，程序员其实挺可爱的，就是有时候分不清现实和代码。（做出敲键盘的动作）要不要听听另一个关于产品经理和程序员的故事？


每次流式输出状态，在这里也就是消息队列，第二次流式输出，由于 ai 回复，增加了 AIMessage，但还是把更新后的状态完整地输出出来

updates 与 values 不同的是，只有状态出现更新时，才会流式输出

例如，如果你有一个调用一次工具的Agent，你应该看到以下更新：
- LLM 节点 ： AIMessage 带有工具调用请求
- 工具节点 ： ToolMessage 带有执行结果
- LLM 节点 ：最终 AI 响应

In [9]:
# 流式模式 = updates
for step in agent.stream(
    {"messages": [{"role": "user", "content": "Tell me a Dad joke"}]},
    stream_mode="updates",
):
    print(step)
    print("-----")

{'model': {'messages': [AIMessage(content='Alright, buckle up for some *dad-level* cheese! 🧀\n\n**Why don\'t skeletons fight each other?**  \n...  \n...  \n**Because they don\'t have the guts!** 💀\n\n*(Leans in with a terrible grin, gives you a thumbs-up, and immediately knocks over a lamp while backing away.)*  \n"Whoops! ...That’s why I stick to *bone*-afide jokes!" 😅', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 91, 'prompt_tokens': 25, 'total_tokens': 116, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'dashscope', 'model_name': 'qwen3-max', 'system_fingerprint': None, 'id': 'chatcmpl-e020efd0-471d-9c6e-ba92-5fe38ca39563', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b540a-440d-73d2-af98-39a0d329e63c-0', usage_metadata={'input_tokens': 25, 'output_tokens': 91, 'total_tokens': 116, 'input_token_details': {'cache_read': 0}, 'output_token_details':

## messages 模式
消息会逐个 token 流式传输，延迟降到最低，非常适合聊天机器人等交互式应用。

使用 messages 流式模式从你的图中的任何部分（包括节点、工具、子图或任务）流式传输大型语言模型（LLM）的输出， 逐个 token。来自 messages 模式的流式输出是一个元组 (message_chunk, metadata)，其中：

- message_chunk：来自 LLM 的 token 或消息片段。
- metadata：一个包含有关图节点和 LLM 调用详情的字典。

In [11]:
for token, metadata in agent.stream(
    {"messages": [{"role": "user", "content": "讲一个关于程序员的笑话"}]},
    stream_mode="messages",
):
    print(f"{token.content}", end="")

为什么程序员总是喜欢用“暗色模式”？

因为他们不想被bug“亮瞎眼”！ 😄

## 工具也可以流式！
流式通常意味着在最终结果准备好之前就把信息传递给用户，这在很多场景中很有用。`get_stream_writer` 允许你轻松地从自己创建的来源中流式发送 `custom` 数据。

In [ ]:
from langchain.agents import create_agent
from langgraph.config import get_stream_writer


def get_weather(city: str) -> str:
    """获取指定城市的天气。"""
    writer = get_stream_writer()        #使用 get_stream_writer 访问流写入器并发出自定义数据。
    # 可流式输出任意自定义数据
    writer(f"查找城市数据: {city}")
    writer(f"请求城市数据: {city}")
    return f"在 {city}总是天晴!"


agent = create_agent(
    model=model,
    tools=[get_weather],
)

for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "SF的天气怎么样?"}]},
    stream_mode=["values", "custom"],
):
    print(chunk)

('values', {'messages': [HumanMessage(content='SF的天气怎么样?', additional_kwargs={}, response_metadata={}, id='515b45b1-3ee7-423a-ae59-2be7945ef832')]})
('values', {'messages': [HumanMessage(content='SF的天气怎么样?', additional_kwargs={}, response_metadata={}, id='515b45b1-3ee7-423a-ae59-2be7945ef832'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 257, 'total_tokens': 278, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'dashscope', 'model_name': 'qwen3-max', 'system_fingerprint': None, 'id': 'chatcmpl-2144b6e9-5f7d-9cc5-a6c7-727dc679c366', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b5410-afd6-7ad1-9295-250813cfe5f8-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'SF'}, 'id': 'call_fc7497e5149442dbadd06441', 'type': 'tool_call'}], usage_metadata={'input_tokens': 257, 'output_tokens': 21, 'total_tokens

In [13]:
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["custom"],
):
    print(chunk)

('custom', '查找城市数据: SF')
('custom', '请求城市数据: SF')


In [14]:
from langchain.agents import create_agent
# 工具端：在工具中使用 runtime.stream_writer 推送流式更新
from langchain.tools import tool, ToolRuntime

@tool
def get_weather(city: str, runtime: ToolRuntime) -> str:
    """获取指定城市的天气（示例带流式进度）。"""
    writer = runtime.stream_writer           # ToolRuntime 提供的流写入器（隐藏于 LLM）
    writer(f"Looking up data for city: {city}")    # 推送实时更新（会出现在 stream 的 custom channel）
    # ...执行耗时操作（HTTP 请求、爬取等）
    writer(f"Acquired data for city: {city}")      # 再次推送进度
    return f"It's always sunny in {city}!"         # 常规返回（会作为 ToolMessage / Tool 的输出被捕获）


agent = create_agent(
    model=model,
    tools=[get_weather],
)

# 调用端：同时订阅 values（完整状态）和 custom（工具 writer 输出）
for mode, chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["values", "custom"],   # 同时请求两类流
):
    if mode == "values":
        # chunk 是当前完整 state 快照（通常包含 messages 列表）
        print("LATEST:", chunk["messages"][-1])
        print(type(chunk["messages"][-1]))
    else:
        # mode == "custom": chunk 是工具 writer 推送的文本（或自定义结构）
        print("TOOL STREAM:", chunk)
    print("-----")

LATEST: content='What is the weather in SF?' additional_kwargs={} response_metadata={} id='4f69d724-28a8-4b21-a54b-309f5c4d3e1e'
<class 'langchain_core.messages.human.HumanMessage'>
-----
LATEST: content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 266, 'total_tokens': 287, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'dashscope', 'model_name': 'qwen3-max', 'system_fingerprint': None, 'id': 'chatcmpl-70382a62-abbf-97ce-9c11-b9650debb66c', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--019b5413-4a82-7ae0-9d85-640bc07a5def-0' tool_calls=[{'name': 'get_weather', 'args': {'city': 'SF'}, 'id': 'call_9979532dddca4321a4a80c04', 'type': 'tool_call'}] usage_metadata={'input_tokens': 266, 'output_tokens': 21, 'total_tokens': 287, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}}
<class 'langchain_core.messages.ai.AI